# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096824


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:45,  3.65s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:15,  2.68s/it]

Rendering models:  13%|█▎        | 4/31 [00:08<00:54,  2.01s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:38,  1.50s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:15<00:00,  3.15it/s]

Rendering models:  94%|█████████▎| 29/31 [00:16<00:00,  2.83it/s]

Rendering models:  97%|█████████▋| 30/31 [00:16<00:00,  2.85it/s]

Rendering models: 100%|██████████| 31/31 [00:17<00:00,  2.89it/s]

ElisabethB                            0.052005
not-logged-in-3b0b929f4475fd276561    0.000229
not-logged-in-0687f3923f81ac7bd40a    0.001266
not-logged-in-673e227f920b24558cc1    0.000296
nurenai                               0.050267
RaeneVigil                            0.001135
Jonas_Cross                           0.000957
jnarayanbvg                           0.000430
not-logged-in-a4cf4d6534661ba8c651    0.001737
not-logged-in-a6e6ceaec621c56b1415    0.019063
Planetme                              0.004845
41Letters                             0.010721
41Letters                             0.000496
not-logged-in-f87600b62b6963cef865    0.045775
Lavadude                              0.025503
ShirleyL                              0.000222
not-logged-in-d55d56ff31d5a738495f    0.003919
DanielEsteves                         0.004356
nicoletraher                          0.004089
tingard                               0.000205
bboyle23                              0.001536
h4ranek      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())